goal: identify under performed gas wells using RailRoad Commision of Texas public data request
methodology: SPE analysis

In [ ]:
import pyodbc 
import pandas as pd
from pandas import DataFrame
from pandas.tools import plotting
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import re
from scipy.spatial.distance import pdist, squareform
import scipy
import glob
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import sys

In [ ]:
con = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
query = "SELECT * FROM INFORMATION_SCHEMA.TABLES"
#query = "SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'"
cursor = con.cursor()
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
con = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
cursor = con.cursor()
string = "DROP TABLE up5_adj_prod"
cursor.execute(string)
con.commit()

In [ ]:
con = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
cursor = con.cursor()
string = "DROP TABLE up5_well_frac_adj"
cursor.execute(string)
con.commit()

# Wellbore locations

In [ ]:
query = "SELECT API, WB_WGS84_LATITUDE, WB_WGS84_LONGITUDE FROM WellBore_Location"
wb_location = pd.read_sql(query,con)
print('wb_location:',list(wb_location.columns))

In [ ]:
wb_location['WB_WGS84_LATITUDE'] = wb_location['WB_WGS84_LATITUDE'].map(lambda x: str(x)[:-1]).astype('float64')/1e6
wb_location['WB_WGS84_LONGITUDE'] = wb_location['WB_WGS84_LONGITUDE'].map(lambda x: str(x)[:-1]).astype('float64')/1e6

In [ ]:
wb_location['API_COUNTY_CODE'] = wb_location['API'].map(lambda x: str(x)[:3])
wb_location['API_UNIQUE_NO'] = wb_location['API'].map(lambda x: str(x)[4:])

In [ ]:
wb_location.head(1)

# API

In [ ]:
query = "SELECT * FROM OG_WELL_COMPLETION"
OG_WELL_COMPLETION = pd.read_sql(query,con)
print('OG_WELL_COMPLETION:',list(OG_WELL_COMPLETION.columns))

In [ ]:
OG_WELL_COMPLETION.head(4)

# Gas production

In [ ]:
query = "SELECT OIL_GAS_CODE, DISTRICT_NO, FIELD_NO, CYCLE_YEAR_MONTH, GAS_WELL_NO, CYCLE_YEAR_MONTH, LEASE_NO, COUNTY_NO, DISTRICT_NO, OPERATOR_NO, LEASE_NAME, CNTY_LSE_GAS_PROD_VOL FROM OG_COUNTY_LEASE_CYCLE"
OG_COUNTY_LEASE_CYCLE = pd.read_sql(query,con)
print('OG_COUNTY_LEASE_CYCLE:',list(OG_COUNTY_LEASE_CYCLE.columns))
OG_COUNTY_LEASE_CYCLE.to_csv('gas_cnty_lse.csv')

In [ ]:
OG_COUNTY_LEASE_CYCLE = pd.read_csv('gas_cnty_lse.csv')

In [ ]:
OG_WELL_COMPLETION.apply(lambda col:pd.to_numeric(col, errors='coerce'))

In [ ]:
OG_WELL_COMPLETION['DISTRICT_NO'] = OG_WELL_COMPLETION['DISTRICT_NO'].astype('int64')
OG_WELL_COMPLETION['LEASE_NO'] = OG_WELL_COMPLETION['LEASE_NO'].map(lambda x: re.sub('[a-zA-Z]', '', x))
OG_WELL_COMPLETION['LEASE_NO'] = OG_WELL_COMPLETION['LEASE_NO'].astype('int64')

In [ ]:
new_og_county_lease_cycle = pd.merge(left = OG_COUNTY_LEASE_CYCLE, right = OG_WELL_COMPLETION, on=['OIL_GAS_CODE', 'DISTRICT_NO', 'LEASE_NO'])

In [ ]:
list(new_og_county_lease_cycle.columns)
new_df = pd.merge(left = new_og_county_lease_cycle, right = wb_location, on =['API_COUNTY_CODE','API_UNIQUE_NO'])

In [ ]:
new_df_g = new_df.loc[new_df['OIL_GAS_CODE'] == 'G']

In [ ]:
new_df_g_s = new_df_g[['API','CYCLE_YEAR_MONTH','DISTRICT_NO','COUNTY_NO','FIELD_NO','OPERATOR_NO','WELL_NO','LEASE_NO','WB_WGS84_LATITUDE','WB_WGS84_LONGITUDE','CNTY_LSE_GAS_PROD_VOL']]

In [ ]:
len(new_df_g_s.API.unique())

In [ ]:
new_df_g_s.to_csv('gas_well_prod.csv')

# rank wells

In [ ]:
new_df_g_s = pd.read_csv('gas_well_prod.csv')

In [ ]:
new_df_g_s.head(2)

In [ ]:
new_df_g_s['WELL_NO'] = new_df_g_s['WELL_NO'].astype('|S')
new_df_g_s.dtypes

In [ ]:
LEASE_list = new_df_g_s.LEASE_NO.unique()
print('total # of leases:', np.shape(LEASE_list))
print('total # of API:', np.shape(new_df_g_s.API.unique()))
lease = random.randint(0, np.shape(LEASE_list)[0])
df_lease = new_df_g_s[new_df_g_s['LEASE_NO'] == LEASE_list[lease]]
print('lease:', df_lease.LEASE_NO.unique())
print('api:', df_lease.API.unique())
print('district:',df_lease.DISTRICT_NO.unique())
print('county:',df_lease.COUNTY_NO.unique())
print('field:',df_lease.FIELD_NO.unique())
print('operator:',df_lease.OPERATOR_NO.unique())
print('well:',df_lease.WELL_NO.unique())
print('lat:',df_lease.WB_WGS84_LATITUDE.unique())
print('lon:',df_lease.WB_WGS84_LONGITUDE.unique())

In [ ]:
#select gas wells that have reporting dates between 199401 and 201312
def cl(df, LEASE_list):
    df_TIME_lease = []
    df_TIME_min = []
    df_TIME_max = []
    df_TIME = pd.DataFrame(columns=['LEASE_NO', 'MIN_TIME', 'MAX_TIME'])
    for ls in range(len(LEASE_list)):
        df_ls = df.loc[df['LEASE_NO'] == LEASE_list[ls], ['LEASE_NO', 'API', 'CYCLE_YEAR_MONTH', 'WB_WGS84_LATITUDE','WB_WGS84_LONGITUDE','CNTY_LSE_GAS_PROD_VOL']]
        df_TIME_lease.append(LEASE_list[ls])
        df_TIME_min.append(df_ls.CYCLE_YEAR_MONTH.min())
        df_TIME_max.append(df_ls.CYCLE_YEAR_MONTH.max())
    df_TIME['LEASE_NO'] = df_TIME_lease
    df_TIME['MIN_TIME'] = df_TIME_min
    df_TIME['MAX_TIME'] = df_TIME_max 
    return df_TIME

In [ ]:
###separate wells to four different parts
#new_df_g_s_p1 = cl(new_df_g_s, LEASE_list[:50000])
#new_df_g_s_p1 = new_df_g_s_p1.dropna()
#new_df_g_s_p2 = cl(new_df_g_s, LEASE_list[50000:100000])
#new_df_g_s_p2 = new_df_g_s_p2.dropna()
#new_df_g_s_p3 = cl(new_df_g_s, LEASE_list[100000:150000])
#new_df_g_s_p3 = new_df_g_s_p3.dropna()
#new_df_g_s_p4 = cl(new_df_g_s, LEASE_list[150000:])
#new_df_g_s_p4 = new_df_g_s_p4.dropna()
#new_df_g_s_p1.to_csv('gas_well_time_p1.csv')
#new_df_g_s_p2.to_csv('gas_well_time_p2.csv')
#new_df_g_s_p3.to_csv('gas_well_time_p3.csv')
#new_df_g_s_p4.to_csv('gas_well_time_p4.csv')

In [ ]:
new_df_g_s_p1 = pd.read_csv('gas_well_time_p1.csv')
new_df_g_s_p2 = pd.read_csv('gas_well_time_p2.csv')
new_df_g_s_p3 = pd.read_csv('gas_well_time_p3.csv')
new_df_g_s_p4 = pd.read_csv('gas_well_time_p4.csv')

In [ ]:
#remove records outside of 199401-201312
new_df_g_s_p14 = pd.concat([new_df_g_s_p1, new_df_g_s_p2, new_df_g_s_p3, new_df_g_s_p4], ignore_index=True)
LEASE_list_199401_201312 = list(new_df_g_s_p14.loc[(new_df_g_s_p14['MIN_TIME'] <= 199401) & (new_df_g_s_p14['MAX_TIME'] >= 201312), 'LEASE_NO'])
new_df_199401_201312 = new_df_g_s.loc[new_df_g_s['LEASE_NO'].isin(LEASE_list_199401_201312)]
new_df_199401_201312 = new_df_199401_201312.loc[(new_df_199401_201312['CYCLE_YEAR_MONTH'] >=199401) & (new_df_199401_201312['CYCLE_YEAR_MONTH'] <= 201312)]

In [ ]:
#select gas wells that not all zeros 
def cl_0(df, LEASE_list):
    lease_list = []
    for ls in range(len(LEASE_list)):
        df_ls = df.loc[df['LEASE_NO'] == LEASE_list[ls]]
        if df_ls[df_ls.CNTY_LSE_GAS_PROD_VOL !=0].empty:
            lease_list.append(LEASE_list[ls])
    return lease_list

In [ ]:
###separate wells into four different parts
LEASE_list_20y_x = cl_0(new_df_199401_201312, LEASE_list_199401_201312[:10000])
LEASE_list_20y_x1 = cl_0(new_df_199401_201312, LEASE_list_199401_201312[10000:25000])
LEASE_list_20y_x2 = cl_0(new_df_199401_201312, LEASE_list_199401_201312[25000:])

In [ ]:
lease_list_20y_x = LEASE_list_20y_x + LEASE_list_20y_x1 + LEASE_list_20y_x2
new_df_20y = new_df_199401_201312.loc[~new_df_199401_201312['LEASE_NO'].isin(lease_list_20y_x)]

In [ ]:
format = '%Y%m'
new_df_20y['Datetime'] = pd.to_datetime(new_df_20y['CYCLE_YEAR_MONTH'], format=format)
new_df_20y = new_df_20y.set_index(pd.DatetimeIndex(new_df_20y['Datetime']))

In [ ]:
#remove outliers
#new_df_20y = new_df_20y[np.abs(new_df_20y.CNTY_LSE_GAS_PROD_VOL - new_df_20y.CNTY_LSE_GAS_PROD_VOL.mean()) <= (3*new_df_20y.CNTY_LSE_GAS_PROD_VOL.std())]

In [ ]:
#new_df_20y.to_csv('new_df_20y.csv')

In [ ]:
new_df_20y = pd.read_csv('new_df_20y.csv')
new_df_20y = new_df_20y.set_index(pd.DatetimeIndex(new_df_20y['Datetime']))
new_df_20y.head(2)

In [ ]:
LEASE_list_20y = list(new_df_20y.LEASE_NO.unique())
np.shape(LEASE_list_20y)

In [ ]:
api_lease = new_df_20y[['API', 'LEASE_NO']].drop_duplicates().reset_index(drop=True)
api_lease_locations = new_df_20y[['API','LEASE_NO','WB_WGS84_LONGITUDE','WB_WGS84_LATITUDE']].drop_duplicates().reset_index(drop=True)

In [ ]:
api_lease_locations.head(2)

# identify adjacents wells for each gas well

In [ ]:
#identify locations for above leases
def loc(df):
    adj_ls = {}
    ls_list = list(df.LEASE_NO.unique())
    for ls in range(0, len(ls_list)):
        adj_ls[ls_list[ls]] = [df.loc[df['LEASE_NO'] == ls_list[ls]].WB_WGS84_LATITUDE.iloc[1], df.loc[df['LEASE_NO'] == ls_list[ls]].WB_WGS84_LONGITUDE.iloc[1]]
    return adj_ls

In [ ]:
new_df_20y_latlon = loc(new_df_20y)
new_df_20y_latlon_df = pd.DataFrame.from_dict(new_df_20y_latlon, orient="index", columns=['lat','lon'])
#new_df_20y_latlon_df.to_csv('new_df_20y_latlon_df.csv')

In [ ]:
new_df_20y_latlon_df.to_csv('new_df_20y_latlon_df.csv')

In [ ]:
#identify adjacents wells for each well within dst
def adj_well(df_dist, dst, ls_list):
    adj_ls_ls = {}
    for ls in range(df_dist.shape[0]):
        adjj = list(df_dist.loc[ls] < dst)
        adj_ls_ls[ls_list[ls]] = [ls_list[i] for i, x in enumerate(adjj) if x]
    return adj_ls_ls

In [ ]:
#dst = 0.02degree ~= 6000 feet radius
#new_df_20y_latlon_df = pd.read_csv('new_df_20y_latlon_df.csv')
dist = pdist(new_df_20y_latlon_df[['lat', 'lon']], 'euclidean')
df_dist = pd.DataFrame(squareform(dist))
adj_well_dic = adj_well(df_dist, 0.03, list(new_df_20y_latlon_df.index.unique()))

In [ ]:
adj_well_dic[131078]
a = new_df_20y_latlon_df[new_df_20y_latlon_df.index == 140808]
b = new_df_20y_latlon_df[new_df_20y_latlon_df.index == 137166]
np.linalg.norm(np.array(a.lat,a.lon)-np.array(b.lat,b.lon))
#scipy.spatial.distance.euclidean(np.array(a.lat,a.lon),np.array(b.lat,b.lon))

# identify under-performed gas wells: cumulative prod indicator

In [ ]:
#20year, 10year, 5year, 4year, 2year, 1year
#LEASE_NO, start_year, end_year, interval_year, cumulative_prod, normalized_prod
def ana_cumulative(df, ls_list):
    df_output = {}
    for ls in range(0, len(ls_list)):
        df_ls = df.loc[df['LEASE_NO'] == ls_list[ls], 'CNTY_LSE_GAS_PROD_VOL']
        df_output[ls_list[ls]] = np.sum(df_ls)
    return df_output

In [ ]:
def ana_int_yr(new_df_20y, yr1, yr2, LEASE_list_20y, adj_well_dic):
    for i in range(len(yr1)):
        new_df_20y_int = new_df_20y[(new_df_20y.index.year >= yr1[i]) & (new_df_20y.index.year <= yr2[i])]
        ana_20y = ana_cumulative(new_df_20y_int, LEASE_list_20y)
        ana_20y_df = pd.DataFrame.from_dict(ana_20y, orient="index")
        ana_20y_df = ana_20y_df.rename(columns={0:'cumulative_prod'})
        lease_list = list(ana_20y_df.index.unique())
        p_list = []
        for ls in range(len(lease_list)):
            ana_20y_df_adj = ana_20y_df.loc[adj_well_dic[lease_list[ls]], 'cumulative_prod']
            p_list.append(pd.qcut(ana_20y_df_adj,100, labels=False, duplicates='drop')[lease_list[ls]])
        ana_20y_df['percentile'] = p_list
        ana_20y_df['start_year'] = yr1[i]
        ana_20y_df['end_year'] = yr2[i]
        ana_20y_df['interval_year'] = yr2[i] - yr1[i] + 1
        ana_20y_df.to_csv('ana_20y_df_' + str(yr1[i]) + '_' + str(yr2[i]) + '.csv')

In [ ]:
#20year
yr1 = [1994]
yr2 = [2013]
ana_int_yr(new_df_20y_nolier, yr1, yr2, LEASE_list_20y, adj_well_dic)
#10year
yr1 = [i for i in range(1994, 2013, 10)]
yr2 = [i for i in range(2003, 2022, 10)]
ana_int_yr(new_df_20y_nolier, yr1, yr2, LEASE_list_20y, adj_well_dic)
#5year
yr1 = [i for i in range(1994, 2014, 5)]
yr2 = [i for i in range(1998, 2018, 5)]
ana_int_yr(new_df_20y_nolier, yr1, yr2, LEASE_list_20y, adj_well_dic)
#4year
yr1 = [i for i in range(1994, 2014, 4)]
yr2 = [i for i in range(1997, 2017, 4)]
ana_int_yr(new_df_20y_nolier, yr1, yr2, LEASE_list_20y, adj_well_dic)
#2year
yr1 = [i for i in range(1994, 2014, 2)]
yr2 = [i for i in range(1995, 2015, 2)]
ana_int_yr(new_df_20y_nolier, yr1, yr2, LEASE_list_20y, adj_well_dic)
#1year
yr1 = [i for i in range(1994, 2014, 1)]
yr2 = [i for i in range(1994, 2014, 1)]
ana_int_yr(new_df_20y, yr1, yr2, LEASE_list_20y, adj_well_dic)

# identify under-performed gas wells: normalized prod indicator

In [ ]:
norm_prod = new_df_20y.groupby('LEASE_NO').apply(lambda x: x.CNTY_LSE_GAS_PROD_VOL.resample('12m').mean())/12
norm_prod_t = norm_prod.T
def ana_normalized(df, ls_list):
    df_output = {}
    for ls in range(0, len(ls_list)):
        df_ls = df.loc[:, ls_list[ls]]
        df_output[ls_list[ls]] = np.mean(df_ls)
    return df_output

In [ ]:
def ana_int_yr_normalized(df, yr1, yr2, ls_list, adj_well_dic):
    for i in range(len(yr1)):
        df_int = df.loc[(df.index.year>=yr1[i]) & (df.index.year>=yr2[i])]
        ana_20y = ana_normalized(df_int, ls_list)
        ana_20y_df = pd.DataFrame.from_dict(ana_20y, orient="index")
        ana_20y_df = ana_20y_df.rename(columns={0:'normalized_prod'})
        lease_list = list(ana_20y_df.index.unique())
        p_list = []
        for ls in range(len(lease_list)):
            ana_20y_df_adj = ana_20y_df.loc[adj_well_dic[lease_list[ls]], 'normalized_prod']
            p_list.append(pd.qcut(ana_20y_df_adj,100, labels=False, duplicates='drop')[lease_list[ls]])
        ana_20y_df['percentile'] = p_list
        ana_20y_df['start_year'] = yr1[i]
        ana_20y_df['end_year'] = yr2[i]
        ana_20y_df['interval_year'] = yr2[i] - yr1[i] + 1
        ana_20y_df.to_csv('ana_20y_normalized_' + str(yr1[i]) + '_' + str(yr2[i]) + '.csv')

In [ ]:
#20year
yr1 = [1994]
yr2 = [2013]
ana_int_yr_normalized(norm_prod_t, yr1, yr2, LEASE_list_20y, adj_well_dic)

In [ ]:
#10year
yr1 = [i for i in range(1994, 2013, 10)]
yr2 = [i for i in range(2003, 2022, 10)]
ana_int_yr_normalized(norm_prod_t, yr1, yr2, LEASE_list_20y, adj_well_dic)

In [ ]:
#5year
yr1 = [i for i in range(1994, 2014, 5)]
yr2 = [i for i in range(1998, 2018, 5)]
ana_int_yr_normalized(norm_prod_t, yr1, yr2, LEASE_list_20y, adj_well_dic)

In [ ]:
#4year
yr1 = [i for i in range(1994, 2014, 4)]
yr2 = [i for i in range(1997, 2017, 4)]
ana_int_yr_normalized(norm_prod_t, yr1, yr2, LEASE_list_20y, adj_well_dic)

In [ ]:
#2year
yr1 = [i for i in range(1994, 2014, 2)]
yr2 = [i for i in range(1995, 2015, 2)]
ana_int_yr_normalized(norm_prod_t, yr1, yr2, LEASE_list_20y, adj_well_dic)

In [ ]:
#1year
yr1 = [i for i in range(1994, 2014, 1)]
yr2 = [i for i in range(1994, 2014, 1)]
ana_int_yr_normalized(norm_prod_t, yr1, yr2, LEASE_list_20y, adj_well_dic)

# performance matrix

In [ ]:
# cumulative prod
allFiles = glob.glob('./' + "ana_20y_df_*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df = df.rename(columns = {'Unnamed: 0': 'lease_no','percentile':'cp_p_'+file_[-13:-4],'cumulative_prod':'cp_'+file_[-13:-4], 'start_year':'cp_y0_'+file_[-13:-4], 'end_year':'cp_y1_'+file_[-13:-4],'interval_year':'cp_yi_'+file_[-13:-4]})
    df = df.set_index('lease_no')
    list_.append(df)
frm = pd.concat([i for i in list_], axis=1)

In [ ]:
filter_col = [col for col in frm if col.startswith('cp_p_')]
frm_ana = frm[filter_col]
lease_list = list(frm_ana.index.unique())
#for ll in range(len(lease_list)):
#    frm.loc[lease_list[ll], 'cp_lt50'] = sum(frm_ana.iloc[ll] <= 50)

In [ ]:
#normalized prod
allFiles = glob.glob('./' + "ana_20y_normalized_*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df = df.rename(columns = {'Unnamed: 0': 'lease_no','percentile':'np_p_'+file_[-13:-4],'cumulative_prod':'np_'+file_[-13:-4], 'start_year':'np_y0_'+file_[-13:-4], 'end_year':'np_y1_'+file_[-13:-4],'interval_year':'np_yi_'+file_[-13:-4]})
    df = df.set_index('lease_no')
    list_.append(df)
frm_norm = pd.concat([i for i in list_], axis=1)

In [ ]:
filter_col = [col for col in frm_norm if col.startswith('np_p_')]
frm_ana_norm = frm_norm[filter_col]
lease_list_norm = list(frm_ana_norm.index.unique())
#for ll in range(len(lease_list_norm)):
#    frm_norm.loc[lease_list_norm[ll], 'np_lt50'] = sum(frm_ana_norm.iloc[ll] <= 50)

In [ ]:
#metrics = frm_ana_norm.join(frm)
#metrics['cp_np'] = metrics['cp_lt50'] + metrics['np_lt50']

In [ ]:
metrics = frm_ana_norm.join(frm_ana)
metrics= metrics.stack().reset_index()
prod_metrics = metrics[['lease_no', 0]]
prod_metrics = prod_metrics.rename(columns = {0:'percentile', 'lease_no':'lease_no'})

In [ ]:
up_well = metrics['cp_np'].nlargest(20).index
op_well = list(set(metrics.index) - set(up_well))

In [ ]:
uw = random.randint(0, len(up_well))
new_df_20y.loc[new_df_20y['LEASE_NO'] == up_well[uw], 'CNTY_LSE_GAS_PROD_VOL'].plot()
adjj = adj_well_dic[up_well[uw]]
ad = random.randint(0, len(adjj))
new_df_20y.loc[new_df_20y['LEASE_NO'] == adjj[ad], 'CNTY_LSE_GAS_PROD_VOL'].plot()

In [ ]:
uw = random.randint(0, len(up_well))
norm_prod_t.loc[:, up_well[uw]].plot()
adjj = adj_well_dic[up_well[uw]]
ad = random.randint(0, len(adjj))
norm_prod_t.loc[:,adjj[ad]].plot()

In [ ]:
new_df_20y.loc[~new_df_20y['LEASE_NO'].isin(up_well), 'CNTY_LSE_GAS_PROD_VOL'].plot(ylim=(0, 7e6))

In [ ]:
norm_prod_t.head(10)
norm_prod_t.loc[:, up_well].plot(legend=None, ylim=(0, 2500))

In [ ]:
norm_prod_t.loc[:, op_well[:50]].plot(legend=None, ylim=(0, 2500))

In [ ]:
op_well[:10]

# push data to sql server as schemas and tables

In [ ]:
#5 tables: 1.prod_metrics, 2.monthly_prod, 3.cum_prod, 4.norm_prod, 5.adjacents, 6.locations
#1.prod_metrics
##metrics['lease_no'] = metrics.index
##metrics_ls = list(metrics.lease_no.unique())
##prod_metrics = metrics[['lease_no', 'cp_np']]
##prod_metrics = prod_metrics.rename(columns = {'cp_np':'cp_np_counts_84', 'lease_no':'lease_no'})
##prod_metrics.cp_np_counts_84 = prod_metrics.cp_np_counts_84.astype('int64')
##prod_metrics.index = range(prod_metrics.shape[0])
##prod_metrics.to_csv('prod_metrics.csv', index=False)

In [ ]:
#1.prod_metrics
prod_metrics.to_csv('prod_metrics.csv', index=False)

In [ ]:
prod_metrics = pd.read_csv('prod_metrics.csv')
metrics_ls = list(prod_metrics.lease_no.unique())

In [ ]:
#2.monthly_prod
monthly_prod = new_df_20y.loc[new_df_20y['LEASE_NO'].isin(metrics_ls), ['LEASE_NO','CYCLE_YEAR_MONTH','CNTY_LSE_GAS_PROD_VOL']]
monthly_prod = monthly_prod.rename(columns = {'LEASE_NO': 'lease_no', 'CYCLE_YEAR_MONTH':'date', 'CNTY_LSE_GAS_PROD_VOL': 'monthly_prod'})
monthly_prod.index = range(monthly_prod.shape[0])
monthly_prod = monthly_prod[np.abs(monthly_prod.monthly_prod - monthly_prod.monthly_prod.mean()) <= (3*monthly_prod.monthly_prod.std())]
monthly_prod.to_csv('monthly_prod.csv', index=False)

In [ ]:
#3.cumulative prod: new_df_20y
#5 years only
yr1 = [i for i in range(1994, 2014, 5)]
yr2 = [i for i in range(1998, 2018, 5)]
def ana_int_yr_5ysum(new_df_20y, yr1, yr2, LEASE_list_20y):
    for i in range(len(yr1)):
        new_df_20y_int = new_df_20y[(new_df_20y.index.year >= yr1[i]) & (new_df_20y.index.year <= yr2[i])]
        ana_20y = ana_cumulative(new_df_20y_int, LEASE_list_20y)
        ana_20y_df = pd.DataFrame.from_dict(ana_20y, orient="index")
        ana_20y_df = ana_20y_df.rename(columns={0:'cumulative_prod'})
        ana_20y_df['start_year'] = yr1[i]
        ana_20y_df['end_year'] = yr2[i]
        ana_20y_df.to_csv('ana_20y_cp_5ysum_' + str(yr1[i]) + '_' + str(yr2[i]) + '.csv')
ana_int_yr_5ysum(new_df_20y, yr1, yr2, LEASE_list_20y)

# cumulative prod
allFiles = glob.glob('./' + "ana_20y_cp_*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df = df.rename(columns = {'Unnamed: 0': 'lease_no','cumulative_prod':'cp_'+file_[-13:-4], 'start_year':'y0_'+file_[-13:-4], 'end_year':'y1_'+file_[-13:-4]})
    df = df.set_index('lease_no')
    list_.append(df)
frm = pd.concat([i for i in list_], axis=1)
filter_col = [col for col in frm if col.startswith('cp_')]
prod_cumul = frm[filter_col]
prod_cumul = prod_cumul.rename(columns = {'cp_1994_1998':'199606', 'cp_1999_2003':'200106', 'cp_2004_2008':'200606','cp_2009_2013':'201106'})
prod_cumul = prod_cumul.stack().reset_index()
prod_cumul = prod_cumul.rename(columns={'level_1':'date', 0:'cp'})
prod_cumul.to_csv('prod_cumul.csv', index=False)

In [ ]:
#4.normalized prod: norm_prod_t
#5 years only
yr1 = [i for i in range(1994, 2014, 5)]
yr2 = [i for i in range(1998, 2018, 5)]
def ana_int_yr_normalized_5ysum(df, yr1, yr2, ls_list):
    for i in range(len(yr1)):
        df_int = df.loc[(df.index.year>=yr1[i]) & (df.index.year>=yr2[i])]
        ana_20y = ana_normalized(df_int, ls_list)
        ana_20y_df = pd.DataFrame.from_dict(ana_20y, orient="index")
        ana_20y_df = ana_20y_df.rename(columns={0:'normalized_prod'})
        ana_20y_df['start_year'] = yr1[i]
        ana_20y_df['end_year'] = yr2[i]
        ana_20y_df.to_csv('ana_20y_np_5ysum_' + str(yr1[i]) + '_' + str(yr2[i]) + '.csv')
ana_int_yr_normalized_5ysum(norm_prod_t, yr1, yr2, LEASE_list_20y)
# normalized prod
allFiles = glob.glob('./' + "ana_20y_np_*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df = df.rename(columns = {'Unnamed: 0': 'lease_no','normalized_prod':'np_'+file_[-13:-4], 'start_year':'y0_'+file_[-13:-4], 'end_year':'y1_'+file_[-13:-4]})
    df = df.set_index('lease_no')
    list_.append(df)
frm = pd.concat([i for i in list_], axis=1)
filter_col = [col for col in frm if col.startswith('np_')]
prod_norm = frm[filter_col]
prod_norm = prod_norm.rename(columns = {'np_1994_1998':'199606', 'np_1999_2003':'200106', 'np_2004_2008':'200606','np_2009_2013':'201106'})
prod_norm = prod_norm.stack().reset_index()
prod_norm = prod_norm.rename(columns={'level_1':'date', 0:'np'})
prod_norm.to_csv('prod_norm.csv', index=False)

In [ ]:
#5.adjacents
adjacents = pd.DataFrame()
adjacents['lease_no'] = adj_well_dic.keys()
adjacents['adjacents'] = adjacents['lease_no'].map(adj_well_dic)
adj_wells = (adjacents['adjacents'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('adjacents').join(adjacents[['lease_no']], how='right'))
adj_wells['adjacents'] = adj_wells['adjacents'].astype('int')
adj_wells.to_csv('adj_wells.csv', index=False)

In [ ]:
#6.locations
locations_wells = new_df_20y_latlon_df
locations_wells['lease_no'] = locations_wells.index
locations_wells = locations_wells.reset_index()
locations_wells = locations_wells.drop(columns='index')
locations_wells.head(4)

In [ ]:
con = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
cursor = con.cursor()
string = "DROP TABLE locations_wells"
cursor.execute(string)
con.commit()

In [ ]:
prod_metrics = pd.read_csv('prod_metrics.csv')
adj_wells = pd.read_csv('adj_wells.csv')
monthly_prod = pd.read_csv('monthly_prod.csv')

In [ ]:
import datetime
prod_cumul = pd.read_csv('prod_cumul.csv')
prod_cumul['date'] = prod_cumul['date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m'))
print(prod_cumul.dtypes)
prod_cumul.head(2)

In [ ]:
prod_norm = pd.read_csv('prod_norm.csv')
prod_norm['date'] = prod_norm['date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m'))
print(prod_norm.dtypes)
prod_norm.head(2)

In [ ]:
params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)
#prod_metrics.to_sql(name='well_metrics',con=engine, if_exists = 'append', chunksize=1000, index=False, schema="dbo")
prod_norm.to_sql(name='prod_norm',con=engine, if_exists = 'replace', chunksize=500, index=False, schema="dbo")
#prod_cumul.to_sql(name='prod_cumul',con=engine, if_exists = 'replace', chunksize=500, index=False, schema="dbo")
#adj_wells.to_sql(name='adj_wells',con=engine, if_exists = 'replace', chunksize=1000, index=False, schema="dbo")
#locations_wells.to_sql(name='well_locations',con=engine, if_exists = 'append', chunksize=500, index=False, schema="dbo")
#monthly_prod.to_sql(name='monthly_prod',con=engine, if_exists = 'replace', chunksize=500, index=False, schema="dbo")

# explore all tables

In [ ]:
query = "SELECT * FROM under_performed_wells"
under_performed_wells_test = pd.read_sql(query,con)
print(under_performed_wells_test.columns)
under_performed_wells_test.head(2)

In [ ]:
up5 = under_performed_wells_test.loc[under_performed_wells_test['percentile']<5].groupby('lease_no').agg('count') >= 84
up5_wells = up5.iloc[[i for i, x in enumerate(up5.lat) if x]].index
up5_wells_top50 = up5_wells[:50]

In [ ]:
query = "SELECT * FROM well_frac"
well_frac_test = pd.read_sql(query,con)
print(well_frac_test.columns)
well_frac_test.head(2)

In [ ]:
query = "SELECT * FROM dbo.adj_wells"
adj_wells_test = pd.read_sql(query,con)
adj_wells_test.head(2)

In [ ]:
up5_wells_top50_adj = adj_wells_test.loc[adj_wells_test['lease_no'].isin(up5_wells_top50)]
up5_wells_top50_adj.head(2)

In [ ]:
query = "SELECT * FROM dbo.monthly_prod"
monthly_prod_test = pd.read_sql(query,con)
monthly_prod_test.columns
monthly_prod_test.head(2)

In [ ]:
up5_top50_adj_prod = pd.merge(left = up5_wells_top50_adj, right = monthly_prod_test, left_on = 'adjacents', right_on = 'lease_no')
up5_top50_adj_prod = up5_top50_adj_prod.drop('lease_no_y', axis=1)
up5_top50_adj_prod = up5_top50_adj_prod.rename(columns={'lease_no_x':'lease_no'})
up5_top50_adj_prod.head(2)

In [ ]:
well_frac_adj = pd.merge(left = well_frac_test, right = adj_wells_test, left_on = 'LEASE_NO', right_on = 'adjacents')
well_frac_adj = well_frac_adj[['lease_no', 'adjacents']].drop_duplicates()
well_frac_adj.head(2)

In [ ]:
# push well_frac table to SQL server
params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)
up5_wells_top50_adj.to_sql(name='up5_wells_top50_adj',con=engine, if_exists = 'replace', chunksize=200, index=False, schema="dbo")
#up5_top50_adj_prod.to_sql(name='up5_top50_adj_prod',con=engine, if_exists = 'replace', chunksize=200, index=False, schema="dbo")
#well_frac_adj.to_sql(name='well_frac_adj',con=engine, if_exists = 'replace', chunksize=200, index=False, schema="dbo")

In [ ]:
ls = random.randint(0, well_frac_adj.shape[0])
print(well_frac_adj.iloc[ls])
up5_frac = monthly_prod_test.loc[monthly_prod_test['lease_no'].isin(well_frac_adj.iloc[ls].values.tolist())]
fig, ax = plt.subplots()
for key, grp in up5_frac.groupby(['lease_no']):
    ax = grp.plot(ax=ax, kind='line', x='date', y='monthly_prod', legend=key)

In [ ]:
query = "SELECT * FROM prod_metrics"
prod_metrics_test = pd.read_sql(query,con)
prod_metrics_test.columns

In [ ]:
query = "SELECT * FROM well_locations"
locations_wells_test = pd.read_sql(query,con)
locations_wells_test.head(2)

In [ ]:
query = "SELECT * FROM dbo.prod_norm"
prod_norm_test = pd.read_sql(query,con)
print(prod_norm_test.dtypes)
prod_norm_test.head(2)

In [ ]:
query = "SELECT * FROM dbo.prod_cumul"
prod_cumul_test = pd.read_sql(query,con)
prod_cumul_test.dtypes
prod_cumul_test.head(2)

In [ ]:
query = "SELECT * FROM OG_COUNTY_LEASE_CYCLE"
OG_COUNTY_LEASE_CYCLE = pd.read_sql(query,con)
print('OG_COUNTY_LEASE_CYCLE:',list(OG_COUNTY_LEASE_CYCLE.columns))
OG_COUNTY_LEASE_CYCLE.head(5)
plt.plot(OG_COUNTY_LEASE_CYCLE.CNTY_LSE_OIL_PROD_VOL)

In [ ]:
query = "SELECT TOP 10 * FROM OG_COUNTY_LEASE_CYCLE_old"
OG_COUNTY_LEASE_CYCLE_old = pd.read_sql(query,con)
print('OG_COUNTY_LEASE_CYCLE_old:',list(OG_COUNTY_LEASE_CYCLE_old.columns))
OG_COUNTY_LEASE_CYCLE_old.head(5)

In [ ]:
query = "SELECT TOP 10 * FROM Wellbore"
Wellbore = pd.read_sql(query,con)
print('columns:',list(Wellbore.columns))
Wellbore.head(5)

In [ ]:
query = "SELECT TOP 100 * FROM FrackJobDates"
FrackJobDates = pd.read_sql(query,con)
print('FrackJobDates:', list(FrackJobDates.columns))
FrackJobDates.head(2)

In [ ]:
query = "SELECT TOP 100 * FROM vFrackJobs"
vFrackJobs = pd.read_sql(query,con)
print('vFrackJobs:', list(vFrackJobs.columns))
vFrackJobs.head(2)

In [ ]:
query = "SELECT * FROM vFrackJobs INNER JOIN FrackingJobs_ProductionResults ON vFrackJobs.API = FrackingJobs_ProductionResults.API"
prod_frac = pd.read_sql(query,con)
print('prod_frac:', list(prod_frac.columns))
prod_frac.head(2)

In [ ]:
query = "SELECT TOP 100 * FROM FrackingJobs_ProductionResults"
FrackingJobs_PR = pd.read_sql(query,con)
print('FrackingJobs_PR', list(FrackingJobs_PR.columns))
FrackingJobs_PR.head(2)

# FracFocus additional information about fracture jobs

In [ ]:
allFiles = glob.glob('./FracFocus/' + "FracFocusRegistry_*.csv")
frame = pd.DataFrame()
list_ = []
fields = ['APINumber','JobStartDate', 'Latitude', 'Longitude', 'JobEndDate','Purpose','IngredientName']
for file_ in allFiles:
    df = pd.read_csv(file_,usecols=fields, header=0)
    list_.append(df)

In [ ]:
frm = pd.concat([i for i in list_], axis=0).drop_duplicates(keep=False)

In [ ]:
frm.head(2)

In [ ]:
len(list(frm.APINumber.unique()))

# retrieve under performed gas wells

In [ ]:
query = "SELECT * FROM dbo.well_metrics"
well_metrics_test = pd.read_sql(query,con)
well_metrics_test.columns

In [ ]:
up = well_metrics_test[well_metrics_test['percentile']<5].groupby('lease_no').percentile.count() == 84
up_wells = up[up.values == True].index.values

In [ ]:
#fracking jobs only include info for ~6000 oil+gas wells, no info for the selected ~40,000 gas well that have records from 1994 to 2013.
prod_frac.loc[prod_frac['LEASE_NO'].isin(well_metrics_test.lease_no)]
len(list(prod_frac.LEASE_NO.unique()))

In [ ]:
up_wells_info = api_lease_locations.loc[api_lease.LEASE_NO.isin(up_wells)]
up_wells_info.head(2)

# University of Texas Land data: sparse las files

# EPA (Envionment Protection Agency): cleaned FracFocus 1.0 voluntarily data from 2011 to 2013

In [ ]:
QaWell = pd.read_csv('QaWell.csv')
print('QaWell columns:', list(QaWell.columns))
QaWell.head(2)

In [ ]:
QaIngredient = pd.read_csv('QaIngredient.csv')
print('QaIngredient columns:', list(QaIngredient.columns))
QaIngredient.head(2)

In [ ]:
well_ingredient = pd.merge(left = QaWell, right=QaIngredient, on='WellId')

In [ ]:
well_ingredient['API'] = well_ingredient['APIFFQA'].apply(lambda x: x[3:])

In [ ]:
api_lease_locations.head(2)

In [ ]:
overlap_well_ingredient = pd.merge(left = api_lease_locations, right = well_ingredient, on = 'API')

In [ ]:
#53 APIs overlapped
len(overlap_well_ingredient.API.unique())

In [ ]:
well_frac = overlap_well_ingredient[['API', 'LEASE_NO', 'LatitudeFFQA', 'LongitudeFFQA', 'DateFFQA', 'TradeName', 'Purpose', 'ChemicalName', 'AdditiveConcentration', 'FluidConcentration']]

In [ ]:
well_frac.groupby('API').head(1)

In [ ]:
# push well_frac table to SQL server
params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)
well_frac.to_sql(name='well_frac',con=engine, if_exists = 'append', chunksize=200, index=False, schema="dbo")

# FracFocus APIs are different from RRC APIs: 16-18 digits vs 8-10 digits, distances were used to match with no luck

In [ ]:
frm.head(2)

In [ ]:
frm['API'] = frm['APINumber'].map(lambda x: str(x)[3:6] + '-' + str(x)[6:11])

In [ ]:
print('# of overlapped APIs RRC & FracFocus:', sum(frm['API'].isin(api_lease_locations.API)))
print('# of overlapped APIs under-performed RRC & FracFocus:', sum(frm['API'].isin(up_wells_info.API)))

In [ ]:
#APIs do not match between frm and up_wells_info
frm_up = frm.loc[frm['API'].isin(up_wells_info.API), ['APINumber', 'API', 'Latitude', 'Longitude']]
frm_up_info = frm_up.merge(up_wells_info, on='API')
frm_up_info.head(3)

In [ ]:
up_wells_info.head(2)

In [ ]:
frm.head(2)

In [ ]:
frm_api_locations = frm[['APINumber', 'Latitude', 'Longitude']].drop_duplicates()

In [ ]:
frm_api_locations.head(2)

In [ ]:
ls = 125
a = up_wells_info[up_wells_info.LEASE_NO == ls]
print(a.API)
i = random.randint(0,(frm_api_locations.shape[0]))
b = frm_api_locations.iloc[i]
print(b.APINumber)  
print((np.abs(a.WB_WGS84_LATITUDE - b.Latitude) < 1e-2))
print((np.abs(-1*a.WB_WGS84_LONGITUDE - b.Longitude) < 1e-2))

In [ ]:
ls = 125
a = up_wells_info[up_wells_info.LEASE_NO == ls]
up_dst = {}
dst_list = []
for i in range(frm_api_locations.shape[0]):
    b = frm_api_locations.iloc[i]
    dst = np.linalg.norm(np.array(a.WB_WGS84_LATITUDE,-a.WB_WGS84_LONGITUDE)-np.array(b.Latitude,b.Longitude))
    dst_list.append(dst)
up_dst[ls] = dst_list

In [ ]:
frm_api_locations.iloc[np.array(up_dst[125]).argmin()]

In [ ]:
a

# Explore tables for Power BI dashboard

In [ ]:
con = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=40.124.30.130;DATABASE=TRC;UID=bpcs;PWD=319f0fa0f2ce18628E4a!')
query = "SELECT * FROM INFORMATION_SCHEMA.TABLES"
#query = "SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'"
cursor = con.cursor()
cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    print(row)

In [ ]:
# well_locaitons, well_metrics, prod_norm, prod_cumul, prod_metrics, monthly_prod, adj_wells, Well_frac

In [ ]:
query = "SELECT * FROM monthly_prod"
monthly_prod = pd.read_sql(query,con)
print(monthly_prod.columns)
monthly_prod.head(2)

In [ ]:
query = "SELECT * FROM adj_wells"
adj_wells = pd.read_sql(query,con)
print(adj_wells.columns)
adj_wells.head(2)

In [ ]:
adj_wells.loc[adj_wells['adjacents'] ==  127494, 'lease_no'].unique()

In [ ]:
adj_131086 = adj_wells.loc[adj_wells['adjacents'] ==  131086, 'lease_no'].unique()

In [ ]:
query = "SELECT * FROM well_frac"
well_frac = pd.read_sql(query,con)
print(well_frac.columns)
well_frac.head(2)

In [ ]:
monthly_prod_frac = pd.merge(left = well_frac, right = monthly_prod, left_on = 'LEASE_NO', right_on = 'lease_no')
monthly_prod_frac.head(2)

In [ ]:
frac_list = monthly_prod_frac.LEASE_NO.unique()

In [ ]:
ls = random.randint(0, len(frac_list))
print(frac_list[ls])
prod = monthly_prod_frac.loc[monthly_prod_frac['LEASE_NO'] == frac_list[ls], ['date','monthly_prod', 'DateFFQA']]
if len(prod.monthly_prod.value_counts()) > 0:
    prod.plot(x='date',y='monthly_prod',marker='.',linestyle='None')
print(prod.DateFFQA.unique())

In [ ]:
# 143739(x), 131086, 127494, 134419(x), 136145

In [ ]:
ls = random.randint(0, len(adj_131086))
print(adj_131086[ls])
prod = monthly_prod_frac.loc[monthly_prod_frac['LEASE_NO'] == adj_131086[ls], ['date','monthly_prod', 'DateFFQA']]
if len(prod.monthly_prod.value_counts()) > 0:
    prod.plot(x='date',y='monthly_prod',marker='.',linestyle='None')
print(prod.DateFFQA.unique())

In [ ]:
prod = monthly_prod_frac.loc[monthly_prod_frac['LEASE_NO'] == 134419, ['date','monthly_prod', 'DateFFQA']]
if len(prod.monthly_prod.value_counts()) > 0:
    prod.plot(x='date',y='monthly_prod',marker='.',linestyle='None')
print(prod.DateFFQA.unique())

In [ ]:
prod = monthly_prod_frac.loc[monthly_prod_frac['LEASE_NO'] == 136145, ['date','monthly_prod', 'DateFFQA']]
if len(prod.monthly_prod.value_counts()) > 0:
    prod.plot(x='date',y='monthly_prod',marker='.',linestyle='None')
print(prod.DateFFQA.unique())

In [ ]:
prod = monthly_prod_frac.loc[monthly_prod_frac['LEASE_NO'] == 127494, ['date','monthly_prod', 'DateFFQA']]
if len(prod.monthly_prod.value_counts()) > 0:
    prod.plot(x='date',y='monthly_prod',marker='.',linestyle='None')
print(prod.DateFFQA.unique())

In [ ]:
prod = monthly_prod_frac.loc[monthly_prod_frac['LEASE_NO'] == 143739, ['date','monthly_prod', 'DateFFQA']]
if len(prod.monthly_prod.value_counts()) > 0:
    prod.plot(x='date',y='monthly_prod',marker='.',linestyle='None')
print(prod.DateFFQA.unique())

In [ ]:
prod = monthly_prod_frac.loc[monthly_prod_frac['LEASE_NO'] == 131086, ['date','monthly_prod', 'DateFFQA']]
if len(prod.monthly_prod.value_counts()) > 0:
    prod.plot(x='date',y='monthly_prod',marker='.',linestyle='None')
print(prod.DateFFQA.unique())